In [9]:
# A Database CLI Application

# Import modules
import pandas as pd
import os
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

def get_closing_prices(symbols, start, end):

    # The Alpaca Parameters for timeframe and daterange
    # `today` is a timestamp using Pandas Timestamp
    # `a_year_ago` is calculated using Pandas Timestamp and Timedelta
    timeframe = '1Day'
    start_date = pd.Timestamp(start, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end, tz="America/New_York").isoformat()

    # The Alpaca tradeapi.REST object
    alpaca = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        api_version="v2")

    # Use the Alpaca get_bars function to get the closing prices for the stocks.
    portfolio_prices_df = alpaca.get_bars(
        symbols,
        timeframe,
        start=start_date,
        end=end_date
    ).df
    
    # For the new closing_prices_df DataFrame, keep only the date component
    # portfolio_prices_df.index = portfolio_prices_df.index.date
    # Reorganize the DataFrame to have a MultiIndex.
    dfs = [
        portfolio_prices_df[portfolio_prices_df['symbol']==symbol].drop('symbol', axis=1)
        for symbol in symbols
    ]
    prices_df_Monte_Carlo = pd.concat([x for x in dfs], axis=1, keys=symbols)
    
    # Create an empty DataFrame for holding the closing prices
    closing_prices_df = pd.DataFrame()
    for symbol in symbols:
        closing_prices_df[symbol] = portfolio_prices_df['close'][portfolio_prices_df['symbol']==symbol]
    
    return (prices_df_Monte_Carlo, closing_prices_df)

symbols1 = ['SQ','SHOP', 'TSLA','NVDA']
symbols2 = ['AAPL', 'GOOG', 'MSFT', 'XOM', 'T']       
symbols3 = ['JPM','TLT','IEF', 'VGSH']   

symbols = symbols1

start = '2021-01-13'
end = '2023-01-13'
Monte_Carlo_df, close_df=get_closing_prices(symbols, start, end)

In [10]:
close_df.tail()

,SQ,SHOP,TSLA,NVDA
timestamp,,,,
2023-01-09 05:00:00+00:00,69.06,37.19,119.77,156.28
2023-01-10 05:00:00+00:00,70.03,36.50,118.85,159.09
2023-01-11 05:00:00+00:00,71.67,36.44,123.22,160.01
2023-01-12 05:00:00+00:00,71.80,36.76,123.56,165.11
2023-01-13 05:00:00+00:00,71.65,38.89,122.40,168.99


In [11]:
Monte_Carlo_df.tail()

SQ                                             \
                             open   high    low  close    volume trade_count   
timestamp                                                                      
2023-01-09 05:00:00+00:00  69.700  70.61  68.44  69.06  12989703      129168   
2023-01-10 05:00:00+00:00  68.675  70.19  67.66  70.03   9057527       83090   
2023-01-11 05:00:00+00:00  70.960  71.89  70.02  71.67   9038565       86531   
2023-01-12 05:00:00+00:00  71.880  72.11  68.11  71.80  11852613      105878   
2023-01-13 05:00:00+00:00  69.810  71.99  69.51  71.65   9330993       85253   

                                       SHOP                ...       TSLA  \
                                vwap   open   high    low  ...     volume   
timestamp                                                  ...              
2023-01-09 05:00:00+00:00  69.506649  37.70  38.49  37.16  ...  190534048   
2023-01-10 05:00:00+00:00  69.236502  36.97  37.40  36.04  ...  167827711   
2023-01-11 05:00:00+00:00  71.133566  36.64  37.35  35.78  ...  183956114   
2023-01-12 05:00:00+00:00  70.693010  37.29  37.40  35.60  ...  169400992   
2023-01-13 05:00:00+00:00  71.039367  35.80  39.07  35.67  ...  180633086   

                                                       NVDA                    \
                          trade_count        vwap      open      high     low   
timestamp                                                                       
2023-01-09 05:00:00+00:00     1746068  121.034846  152.8400  160.5600  151.41   
2023-01-10 05:00:00+00:00     1405221  117.840067  155.0700  159.6199  154.72   
2023-01-11 05:00:00+00:00     1513411  122.678008  158.3951  160.2800  155.63   
2023-01-12 05:00:00+00:00     1432274  121.037246  161.0000  166.3700  154.92   
2023-01-13 05:00:00+00:00     1429419  119.475409  162.7800  169.2200  161.65   

                                                                     
                            close    volume trade_count        vwap  
timestamp                                                            
2023-01-09 05:00:00+00:00  156.28  50441978      518824  157.877164  
2023-01-10 05:00:00+00:00  159.09  38431016      361296  157.470753  
2023-01-11 05:00:00+00:00  160.01  35362507      339554  158.238363  
2023-01-12 05:00:00+00:00  165.11  55154070      531718  162.031053  
2023-01-13 05:00:00+00:00  168.99  44714211      426051  166.757580  

[5 rows x 28 columns]